# Analysis of Stable and Unstable Manifolds

In this notebook, we will analyze a nonlinear dynamical system and compute its stable and unstable manifolds near hyperbolic equilibrium points.

## System of ODEs

We consider the following system of ordinary differential equations:

$$\begin{aligned}
\frac{dx}{dt} &= f_{1}(x,y) \\
\frac{dy}{dt} &= f_{2}(x,y)
\end{aligned}$$

Hopefully, the code is designed to handle any polynomial system of ODEs. Fell free to test whatever you want !

Our goal is to:
1. Find the equilibrium points
2. Identify hyperbolic equilibrium points
3. Compute the stable and unstable manifolds
4. Visualize the results

## Setup: Package Installation and Loading

First, we need to set up our environment with the necessary Julia packages.

In [ ]:
using Pkg;
Pkg.activate(@__DIR__);
Pkg.resolve()
Pkg.update()
Pkg.instantiate(); # Run this line if you have some issue loading packages

  Activating project at `~/Documents/LPHYS2114/LPHYS2114_texfiles_french/Serie 4/notebooks`
     Project No packages added to or removed from `~/Documents/LPHYS2114/LPHYS2114_texfiles_french/Serie 4/notebooks/Project.toml`
    Manifest No packages added to or removed from `~/Documents/LPHYS2114/LPHYS2114_texfiles_french/Serie 4/notebooks/Manifest.toml`
    Updating registry at `~/.julia/registries/General.toml`
   Installed Static ────────── v1.3.1
   Installed SciMLLogging ──── v1.3.1
   Installed BlockArrays ───── v1.9.0
   Installed SciMLBase ─────── v2.123.0
   Installed ModelingToolkit ─ v10.26.1
   Installed LinearSolve ───── v3.28.0
   Installed OrdinaryDiffEq ── v6.103.0
    Updating `~/Documents/LPHYS2114/LPHYS2114_texfiles_french/Serie 4/notebooks/Project.toml`
  [961ee093] ↑ ModelingToolkit v10.26.0 ⇒ v10.26.1 [loaded: v10.26.0]
  [1dea7af3] ↑ OrdinaryDiffEq v6.102.1 ⇒ v6.103.0 [loaded: v6.102.1]
    Updating `~/Documents/LPHYS2114/LPHYS2114_texfiles_french/Serie 4/notebooks

In [2]:
using Symbolics, Groebner # Groebner engine is needed for symbolic_solve with multiple variables
# If the system is more complex, consider using SymPy or evern more powerful engine such as WolframEngine (Mathematica)
using ModelingToolkit, OrdinaryDiffEq
using Makie, CairoMakie
set_theme!(theme_latexfonts())

using LinearAlgebra

## Step 1: Define the Symbolic Variables and ODE System

We use symbolic computation to define our system. The `@independent_variables` and `@variables` macros allow us to work with symbolic expressions.

In [ ]:
# Define the symbolic variables for the ODE system
@independent_variables t
@variables x(t) y(t) # Don't specify type, wait for SymbolicUtils v4.0 for Type-stable variables
dₜ = Differential(t);
∂ₓ = Differential(x);
∂ᵧ = Differential(y);

In [ ]:
# Define the system of ODEs
# Note: The Grobner engine cannot handle non-polynomial equations, so we keep everything polynomial
eqs = [
	dₜ(x) ~ x*(1+y) + x^2*y + y^3 + 0//1,
	dₜ(y) ~ -y - x^2 + 0//1,
]

## Step 2: Find Equilibrium Points

Equilibrium points occur where the time derivatives are zero, i.e., where:
$$\frac{dx}{dt} = 0 \quad \text{and} \quad \frac{dy}{dt} = 0$$

We solve this system symbolically and filter for real solutions only.

In [ ]:
# Solve the RHS == 0 for equilibrium points
nl_eqs = [0 ~ eq.rhs for eq in eqs]
roots = symbolic_solve(nl_eqs, [x, y]; dropmultiplicity=true)
# Filter only real solutions
roots = [root for root in roots if all(isreal.(values(root)))]
println("Found $(length(roots)) real equilibrium points:")
for root in roots
    println("  (x, y) = ($(root[x]), $(root[y]))")
end;

## Step 3: Compute the Jacobian and Identify Hyperbolic Points

The **Jacobian matrix** at an equilibrium point determines the local linear behavior:

$$J = \begin{pmatrix}
\frac{\partial f_1}{\partial x} & \frac{\partial f_1}{\partial y} \\
\frac{\partial f_2}{\partial x} & \frac{\partial f_2}{\partial y}
\end{pmatrix}$$

A point is **hyperbolic** if all eigenvalues of the Jacobian have non-zero real parts. For a 2D system, a hyperbolic point is a **saddle point** if the eigenvalues have opposite signs (i.e., their product is negative) and are therefore real (because in 2D, complex eigenvalues come in conjugate pairs).

In [ ]:
# Compute the jacobian matrix
J = vcat([[expand_derivatives(∂ₓ(eq.rhs)) expand_derivatives(∂ᵧ(eq.rhs))] for eq in eqs]...) .|> simplify .|> Num
println("Jacobian matrix:")
display(J)

In [ ]:
# Evaluate the jacobian at each equilibrium point and compute eigenvalues
hyperbolic_points = Any[]
for root in roots
	J_eval = substitute.(J, Ref(root)) .|> Symbolics.symbolic_to_float .|> Float64
	println("Equilibrium point: (x, y) = ($(root[x]), $(root[y]))")
	println("Jacobian at this point:")
	display(J_eval)
	eigs = eigen(J_eval)
	println("Eigenvalues: $(eigs.values)\n")

	if all(isreal.(eigs.values)) && (prod(eigs.values) < 0)
		push!(hyperbolic_points, root)
	end
end
println("$(length(hyperbolic_points)) hyperbolic equilibrium points found:")
for hp in hyperbolic_points
	println("  (x, y) = ($(hp[x]), $(hp[y]))")
end

## Step 4: Select a Hyperbolic Point for Analysis

Now we will analyze one of the hyperbolic points in detail. Change `id_point` to analyze a different hyperbolic equilibrium point.

In [ ]:
id_point::Int = 1 # Between 1 and length(hyperbolic_points)
@assert 1 <= id_point <= length(hyperbolic_points) "id_point must be between 1 and $(length(hyperbolic_points))"
hp = hyperbolic_points[id_point]
J_hp = substitute.(J, Ref(hp)) .|> Symbolics.value .|> Num
println("Selected hyperbolic point: (x, y) = ($(hp[x]), $(hp[y]))")

## Step 5: Coordinate Transformation

To simplify the analysis, we perform two coordinate transformations:

1. **Translation**: Move the hyperbolic point to the origin
   - New coordinates: $(\tilde{x}, \tilde{y}) = (x - x_0, y - y_0)$

2. **Rotation to eigenbasis**: Express the system in terms of the eigenvectors of the Jacobian
   - New coordinates: $(u, v) = P^{-1}(\tilde{x}, \tilde{y})$ where $P$ is the matrix of eigenvectors
   
In these coordinates, the linearized system becomes diagonal:
$$\frac{d}{dt}\begin{pmatrix}u \\ v\end{pmatrix} = \begin{pmatrix}\lambda_1 & 0 \\ 0 & \lambda_2\end{pmatrix}\begin{pmatrix}u \\ v\end{pmatrix} + \text{nonlinear terms}$$

Fortunately, the `eigen` function automatically sort the eigenvalues and eigenvectors in ascending order.

In [ ]:
# Translate the system so that the hyperbolic point is at the origin
@variables x̄ ȳ # Translated variables
translation = [hp[x], hp[y]] .|> Rational
tmap = Dict([x, y] .=> [x̄, ȳ] .+ translation)
invtmap = Dict([x̄, ȳ] .=> [x, y] .- translation)

new_eqs_bar = [
	substitute(eq.rhs, tmap) |> simplify |> expand for eq in eqs
]

In [ ]:
# Express the system in the eigenbasis of the Jacobian
@variables u v # Variables in the eigenbasis

# To avoid loosing Float64 precision when computing eigenvectors, we will try a symbolic solution
@variables λ
λ₁, λ₂ = symbolic_solve(det(J_hp - λ*I) ~ 0, [λ]; dropmultiplicity=false) .|> Num |> sort
display((λ₁, λ₂))

# Compute eignevectors symbolically associated to each eigenvalue
@variables symv₁[1:2] symv₂[1:2]
sol_symv₁ = symbolic_solve(((J_hp - λ₁*I) * collect(symv₁)) .~ 0, collect(symv₁); dropmultiplicity=true)
sol_symv₂ = symbolic_solve(((J_hp - λ₂*I) * collect(symv₂)) .~ 0, collect(symv₂); dropmultiplicity=true)

v₁ = substitute.(substitute.(collect(symv₁), Ref(merge(sol_symv₁...))), Ref(Dict(collect(symv₁) .=> Num[1, 1])))
v₂ = substitute.(substitute.(collect(symv₂), Ref(merge(sol_symv₂...))), Ref(Dict(collect(symv₂) .=> Num[1, 1])))

display(v₁)
display(v₂)

P = hcat(v₁, v₂)
display(P)
invP = inv(P)
display(invP)

# Safety check :
D = invP * J_hp * P # Should be diagonal
display(D)


In [ ]:
map = Dict([u, v] .=> invP * [x̄, ȳ])
inv_map = Dict([x̄, ȳ] .=> P * [u, v])

display(map)
display(inv_map)

In [ ]:
# Express the ODEs in the new (u, v) coordinates
new_eqs = invP * [
	substitute(new_eq_bar, inv_map) for new_eq_bar in new_eqs_bar
] .|> expand

println("System in (u, v) coordinates:")
display(new_eqs)

## Step 6: Compute Local Manifolds via Taylor Expansion

For a hyperbolic saddle point, we have:
- **Stable manifold**: trajectories that approach the equilibrium as $t \to +\infty$
- **Unstable manifold**: trajectories that approach the equilibrium as $t \to -\infty$

In the eigenbasis $(u, v)$, assuming $\lambda_u < 0 < \lambda_v$:
- The **stable manifold** is locally a curve $v = s(u)$ with $s(0) = 0$, $s'(0) = 0$
- The **unstable manifold** is locally a curve $u = h(v)$ with $h(0) = 0$, $h'(0) = 0$

We compute these as Taylor series: $s(u) = s_2 u^2 + s_3 u^3 + \cdots$ and $h(v) = h_2 v^2 + h_3 v^3 + \cdots$

The coefficients are determined by the invariance condition: points on the manifold must flow along the manifold.

In [ ]:
# Set the order of the Taylor expansion
M::Int = 12 # Order of the taylor expansion
@variables s[2:M] h[2:M]

# Define the Taylor series for the stable and unstable manifolds
s_expr = sum(s[k] * u^k for k in 2:M);
s′_expr = sum(s[k] * k * u^(k-1) for k in 2:M);
h_expr = sum(h[k] * v^k for k in 2:M);
h′_expr = sum(h[k] * k * v^(k-1) for k in 2:M);

display(s_expr)
display(h_expr)

### Stable Manifold Computation

For the stable manifold $v = s(u)$, we use the chain rule:
$$\frac{dv}{dt} = \frac{ds}{du} \frac{du}{dt} = s'(u) \cdot f_u(u, s(u))$$

This must equal $f_v(u, s(u))$ for the curve to be invariant. Matching coefficients of powers of $u$ gives us equations for the $s_k$ coefficients.

In [ ]:
# Stable manifold: dv/dt = ds/dt = (ds/du)(du/dt) = s'(u) * new_eqs[1] (evaluated at v = s(u))
lhs_stable = s′_expr * substitute(new_eqs[1], Dict(v => s_expr)) |> expand
rhs_stable = substitute(new_eqs[2], Dict(v => s_expr)) |> expand
coeffs_stable_lhs = Symbolics.coeff.(Ref(lhs_stable), u .^ (2:M))
coeffs_stable_rhs = Symbolics.coeff.(Ref(rhs_stable), u .^ (2:M))
eqs_stable = coeffs_stable_lhs .~ coeffs_stable_rhs

### Unstable Manifold Computation

Similarly, for the unstable manifold $u = h(v)$:
$$\frac{du}{dt} = \frac{dh}{dv} \frac{dv}{dt} = h'(v) \cdot f_v(h(v), v)$$

In [ ]:
# Unstable manifold: du/dt = dh/dt = (dh/dv)(dv/dt) = h'(v) * new_eqs[2] (evaluated at u = h(v))
lhs_unstable = h′_expr * substitute(new_eqs[2], Dict(u => h_expr)) |> expand
rhs_unstable = substitute(new_eqs[1], Dict(u => h_expr)) |> expand
coeffs_unstable_lhs = Symbolics.coeff.(Ref(lhs_unstable), v .^ (2:M))
coeffs_unstable_rhs = Symbolics.coeff.(Ref(rhs_unstable), v .^ (2:M))
eqs_unstable = coeffs_unstable_lhs .~ coeffs_unstable_rhs

In [ ]:
# Helper function to solve the system of equations iteratively
# We solve for each coefficient starting from the lowest order, substituting previously found coefficients
function my_solve(coeffs, vars)
	sol = Dict{Num, Rational}()
	for (i, coeff) in enumerate(coeffs)
		eq = substitute(coeff, sol)
		var = vars[i]
		single_sol = symbolic_linear_solve(eq, var)
		@assert length(single_sol) == 1 "Expected a unique solution for variable $var"
		sol[var] = single_sol[1] |> Symbolics.value |> Rational{BigInt}
	end
	return sol
end;

In [ ]:
# Solve for the coefficients of the stable manifold
stable_manifold_sol = my_solve(eqs_stable, Symbolics.scalarize(s[2:M]))
display(stable_manifold_sol)
stable_manifold_u = [u, substitute(s_expr, stable_manifold_sol)]

println("Local stable manifold (in (u,v) coordinates): v = s(u) = ")
display(stable_manifold_u[2])

In [ ]:
# Solve for the coefficients of the unstable manifold
unstable_manifold_sol = my_solve(eqs_unstable, Symbolics.scalarize(h[2:M]))
display(unstable_manifold_sol)
unstable_manifold_v = [substitute(h_expr, unstable_manifold_sol), v]

println("Local unstable manifold (in (u,v) coordinates): u = h(v)")
println("First few terms:")
display(unstable_manifold_v[1])

In [ ]:
# Create functions for numerical evaluation
stab(u) = substitute(stable_manifold_u[2], Dict(stable_manifold_u[1] => u)) |> Symbolics.value;
ustab(v) = substitute(unstable_manifold_v[1], Dict(unstable_manifold_v[2] => v)) |> Symbolics.value;

## Step 7: Transform Back to Original Coordinates

Now we transform the manifolds back to the original $(x, y)$ coordinate system for visualization.

In [ ]:
# Express the manifolds in the original (x,y) coordinates
stable_manifold_xy = Num.(P) * substitute.(substitute.(stable_manifold_u, Ref(map)), Ref(invtmap)) .+ translation .|> simplify 
display(stable_manifold_xy)
unstable_manifold_xy = Num.(P) * substitute.(substitute.(unstable_manifold_v, Ref(map)), Ref(invtmap)) .+ translation .|> simplify
display(unstable_manifold_xy)

## Step 8: Visualization - Local Manifolds

We now visualize the flow field and the computed local manifolds (from Taylor series).

In [ ]:
# Define the ODE function for plotting
odeSol(_x, _y) = Point2(substitute.([eq.rhs for eq in eqs], Ref(Dict(x => _x, y => _y))));

In [ ]:
# Figure styling parameters
begin
	dx, dy = 4.0, 4.0
	figsize = (600, 600)
	color_sable = colorant"#0E79B2"
	color_unstable = colorant"#F39237"
	background_color = colorant"#262626"
	colormap = cgrad(:batlowW, rev = true)
	leg_bg_color = (colorant"#FBFEF9", 0.85)
end;

In [ ]:
# First time to plot can take a while on Julia...
fig = let
	fig = Figure(size = figsize)
	ax = Axis(fig[1, 1], xlabel = L"x", ylabel = L"y", backgroundcolor = background_color)
	streamplot!(ax, odeSol, (hp[x] - dx) .. (hp[x] + dx), (hp[y] - dy) .. (hp[y] + dy), colormap = colormap,
		gridsize = (32, 32), arrow_size = 8)
	
	# Plot stable manifold
	us_stable = range(hp[x] - 5*dx, hp[x] + 5*dx, length = 750) |> collect
	vs_stable = stab.(us_stable)
	xs_stable, ys_stable = eachrow(P * [us_stable'; vs_stable'] .+ translation)
	lines!(ax, Symbolics.value.(xs_stable), Symbolics.value.(ys_stable), color = color_sable, linewidth = 4, label = L"Stable Manifold $\mathcal{O}\left(u^{%$M}\right)$")
	
	# Plot unstable manifold
	vs_unstable = range(hp[y] - 2*dy, hp[y] + 2*dy, length = 500) |> collect
	us_unstable = ustab.(vs_unstable)
	xs_unstable, ys_unstable = eachrow(P * [us_unstable'; vs_unstable'] .+ translation)
	lines!(ax, Symbolics.value.(xs_unstable), Symbolics.value.(ys_unstable), color = color_unstable, linewidth = 4, label = L"Unstable Manifold $\mathcal{O}\left(v^{%$M}\right)$")
	
	xlims!(ax, hp[x] - dx, hp[x] + dx)
	ylims!(ax, hp[y] - dy, hp[y] + dy)
	# axislegend(ax; position = :rt, backgroundcolor = leg_bg_color)
	fig
end

## Step 9: "Exact" Manifolds via Numerical Integration

To verify our Taylor expansion and see the global structure, we compute the exact manifolds by numerically integrating trajectories that start very close to the hyperbolic point on the local manifolds.

**Method**:
- Start at a point very close to the equilibrium on the stable manifold
- Integrate **backwards** in time (unstable direction becomes stable backwards)
- Start at a point on the unstable manifold
- Integrate **forwards** in time

In [ ]:
# Define the ODE system in ModelingToolkit for numerical integration
@mtkcompile mtk_sys = System(eqs, t)
prob = ODEProblem(mtk_sys, [x => hp[x], y => hp[y]], (0.0, 1.0); jac = true) # Default starting point and time span, we won't use them

In [ ]:
# Numerically integrate trajectories on the exact manifolds

# In this code, type stability should be such that the ODE solvers works in BigFloat, much better than Float64 but slower
sol_stable_left, sol_stable_right, sol_unstable_bottom, sol_unstable_top = let
	δ = 1e-15 # Very small displacement from equilibrium
	alg = Rodas5P()
	solkwargs = (abstol = 1e-15, reltol = 1e-15, saveat = 0.025)
	stable_tspan = (0.0, -125.0)   # Integrate backwards in time
	unstable_tspan = (0.0, 125.0)  # Integrate forwards in time

	# Stable manifold: start at (u, s(u)) with u close to 0
	u_left = -δ
	u_right = δ
	v_left = stab(u_left)
	v_right = stab(u_right)
	xy0_stable_left = P * [u_left, v_left] .+ translation .|> Symbolics.value
	xy0_stable_right = P * [u_right, v_right] .+ translation .|> Symbolics.value

	prob_stable_left = remake(prob, u0 = Dict([x, y] .=> xy0_stable_left))
	prob_stable_right = remake(prob, u0 = Dict([x, y] .=> xy0_stable_right))

	sol_stable_left = solve(prob_stable_left, alg; solkwargs..., tspan = stable_tspan)
	sol_stable_right = solve(prob_stable_right, alg; solkwargs..., tspan = stable_tspan)

	# Unstable manifold: start at (h(v), v) with v close to 0
	v_bottom = -δ
	v_top = δ
	u_bottom = ustab(v_bottom)
	u_top = ustab(v_top)
	xy0_unstable_bottom = P * [u_bottom, v_bottom] .+ translation .|> Symbolics.value
	xy0_unstable_top = P * [u_top, v_top] .+ translation .|> Symbolics.value

	prob_unstable_bottom = remake(prob, u0 = Dict([x, y] .=> xy0_unstable_bottom))
	prob_unstable_top = remake(prob, u0 = Dict([x, y] .=> xy0_unstable_top))

	sol_unstable_bottom = solve(prob_unstable_bottom, alg; solkwargs..., tspan = unstable_tspan)
	sol_unstable_top = solve(prob_unstable_top, alg; solkwargs..., tspan = unstable_tspan)

	(sol_stable_left, sol_stable_right, sol_unstable_bottom, sol_unstable_top)
end;

## Step 10: Visualization - Exact Manifolds

Finally, we visualize the exact stable and unstable manifolds obtained through numerical integration.

In [ ]:
fig_exact = let
	fig = Figure(size = figsize)
	ax = Axis(fig[1, 1], xlabel = L"x", ylabel = L"y", backgroundcolor = background_color)
	streamplot!(ax, odeSol, (hp[x] - dx) .. (hp[x] + dx), (hp[y] - dy) .. (hp[y] + dy), colormap = colormap,
		gridsize = (32, 32), arrow_size = 8)
	
	# Plot stable manifold trajectories
	lines!(ax, sol_stable_left[x, :], sol_stable_left[y, :], color = color_sable, linewidth = 4, label = "Stable Manifold (Exact)")
	lines!(ax, sol_stable_right[x, :], sol_stable_right[y, :], color = color_sable, linewidth = 4)
	
	# Plot unstable manifold trajectories
	lines!(ax, sol_unstable_bottom[x, :], sol_unstable_bottom[y, :], color = color_unstable, linewidth = 4, label = "Unstable Manifold (Exact)")
	lines!(ax, sol_unstable_top[x, :], sol_unstable_top[y, :], color = color_unstable, linewidth = 4)
	
	xlims!(ax, hp[x] - dx, hp[x] + dx)
	ylims!(ax, hp[y] - dy, hp[y] + dy)
	axislegend(ax; position = :rt, backgroundcolor = leg_bg_color)
	fig
end